In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [2]:
import tensorflow as tf
from tensorflow import keras

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numerapi

In [7]:
DOWNLOAD_DATA = napi.check_new_round()
napi = numerapi.NumerAPI()

In [17]:
NUMBER = napi.get_competitions()[0]['number']

2020-02-23 15:34:59,590 INFO numerapi.numerapi: getting rounds...


In [4]:
if DOWNLOAD_DATA:
    napi = numerapi.NumerAPI()
    napi.download_current_dataset(unzip=True)


In [20]:
validate_path =  "numerai_dataset_%d/numerai_tournament_data.csv"%NUMBER
train_path = "numerai_dataset_%d/numerai_training_data.csv"%NUMBER
target = 'target_kazutsugi'
features = [c for c in pd.read_csv(train_path, nrows=1) if c.startswith('feature')]

In [7]:
data_type = {k:np.float16 for k in features}
train_data = pd.read_csv(train_path, dtype=data_type)
validate_data = pd.read_csv(validate_path, dtype=data_type)

In [8]:
def factorize_era(train, test):
    combined_data = pd.concat([train, test])
    # create mask to split data later
    mask = combined_data['data_type'] == 'train'
    combined_data['factorized_era'] = pd.factorize(combined_data['era'])[0]
    df1 = combined_data[mask]
    df2 = combined_data[~mask]
    return df1, df2

In [9]:
train_data, validate_data = factorize_era(train_data, validate_data)

In [10]:
eras = train_data.factorized_era

In [ ]:
checkpoint_path = "autoencoder/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [ ]:
# ceate a callback that saves model state
cp_callback = keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True,verbose=1,period=5)

In [11]:
def train_model(train_data, hidden_layer_size =128, epochs=10):
    # train and return a model for hidden states
    input_layer = keras.layers.Flatten(input_shape =(310,))
    hidden_layer = keras.layers.Dense(hidden_layer_size, activation='sigmoid')
    output_layer = keras.layers.Dense(310)
    model = keras.Sequential([input_layer, hidden_layer, output_layer])
    model.compile('sgd', loss=tf.keras.losses.MeanSquaredError())
    model.load_weights(checkpoint_path)
    model.fit(train_data, train_data, epochs=epochs, callbacks=[cp_callback])
    model2 = keras.Sequential([input_layer, 
                          keras.layers.Dense(hidden_layer_size, activation='sigmoid', weights=model.layers[1].get_weights())
                          ])
    return model2
    
    

In [12]:
model = train_model(train_data[features],hidden_layer_size =128, epochs=20)

Train on 501808 samples
Epoch 1/20
501808/501808 [==============================] - 15s 30us/sample - loss: 0.1194
Epoch 2/20
501808/501808 [==============================] - 15s 30us/sample - loss: 0.1025
Epoch 3/20
501808/501808 [==============================] - 15s 30us/sample - loss: 0.0921
Epoch 4/20
501808/501808 [==============================] - 15s 30us/sample - loss: 0.0839
Epoch 5/20
501808/501808 [==============================] - 15s 30us/sample - loss: 0.0779
Epoch 6/20
501808/501808 [==============================] - 15s 30us/sample - loss: 0.0732
Epoch 7/20
501808/501808 [==============================] - 15s 30us/sample - loss: 0.0693
Epoch 8/20
501808/501808 [==============================] - 15s 31us/sample - loss: 0.0660
Epoch 9/20
501808/501808 [==============================] - 15s 30us/sample - loss: 0.0631
Epoch 10/20
501808/501808 [==============================] - 15s 30us/sample - loss: 0.0606
Epoch 11/20
501808/501808 [==============================] - 15s 

In [52]:
hidden_states = model2.predict(train_data[features])

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,0.686546,0.403259,0.515534,0.440555,0.270163,0.637060,0.791451,0.517351,0.398814,0.680368,...,0.789193,0.386265,0.500714,0.358889,0.774653,0.650972,0.595357,0.619792,0.397901,0.700685
1,0.678058,0.440137,0.526563,0.562909,0.483487,0.751886,0.743556,0.314800,0.465179,0.684957,...,0.789301,0.501485,0.558652,0.337888,0.721919,0.528023,0.594996,0.298944,0.523178,0.739139
2,0.744519,0.377992,0.422764,0.699192,0.283279,0.569649,0.700332,0.353558,0.620805,0.711650,...,0.874467,0.878283,0.334499,0.504314,0.780726,0.445807,0.371922,0.318145,0.383779,0.555009
3,0.768294,0.402401,0.338873,0.725071,0.350228,0.269183,0.636537,0.650944,0.418437,0.593439,...,0.701766,0.247557,0.320002,0.366264,0.836596,0.398537,0.594427,0.634223,0.497441,0.400598
4,0.319311,0.358944,0.627367,0.772601,0.355377,0.739240,0.668478,0.503226,0.238712,0.682116,...,0.560791,0.730925,0.325595,0.379998,0.602860,0.715927,0.533817,0.590468,0.382250,0.567248
5,0.771958,0.626780,0.378768,0.576668,0.469040,0.345335,0.577187,0.535539,0.428005,0.659645,...,0.642748,0.511916,0.347127,0.651098,0.750937,0.426077,0.558014,0.366174,0.521878,0.720641
6,0.737905,0.258073,0.651299,0.354297,0.506396,0.920827,0.522199,0.200624,0.420622,0.745775,...,0.800930,0.717842,0.444556,0.405858,0.833933,0.612171,0.587907,0.290218,0.361943,0.778849
7,0.467896,0.425218,0.712249,0.213824,0.162465,0.495681,0.525734,0.357414,0.732705,0.753445,...,0.628675,0.471738,0.901142,0.550375,0.298415,0.540434,0.648483,0.607942,0.535389,0.778964
8,0.577043,0.699948,0.183816,0.415482,0.360656,0.277488,0.543861,0.366436,0.700954,0.456649,...,0.429175,0.695213,0.533954,0.666604,0.367624,0.493833,0.458778,0.291491,0.560173,0.647327
9,0.350947,0.638715,0.536195,0.496080,0.241711,0.349899,0.401692,0.480463,0.651086,0.669102,...,0.617950,0.374989,0.853920,0.672477,0.558304,0.577380,0.582090,0.386625,0.398771,0.703467
